In [1]:
!pip install torch transformers datasets trl peft wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 92.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset, Dataset
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM
from typing import cast
from peft.mapping import get_peft_config, get_peft_model
from peft.utils.peft_types import TaskType
from peft.tuners.lora.config import LoraConfig
import wandb

MODEL_NAME = "facebook/opt-350m"
DATASET_NAME = "lucasmccabe-lmi/CodeAlpaca-20k"

# LoRA 파라미터
# lora_r: int = 8
# lora_r: int = 128
lora_r: int = 256
lora_dropout: float = 0.1
lora_alpha: int = 32

project_name = "LoRA-instruction-tuning"
wandb.init(project=project_name, name=f"LoRA-rank-{lora_r}")

dataset = cast(Dataset, load_dataset(DATASET_NAME, split="train"))
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# 데이터셋의 상위 95%에 해당하는 최대 토큰 길이: 276
MAX_LENGTH = 300
tokenizer.model_max_length = MAX_LENGTH


def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example["instruction"])):
        text = f"### Question: {example['instruction'][i]}\n ### Answer: {example['output'][i]}"
        output_texts.append(text)
    return output_texts


response_template = " ### Answer:"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

target_modules = set()

for name, module in model.named_modules():
    if isinstance(module, torch.nn.Linear):
        names = name.split(".")
        target_modules.add(names[0] if len(names) == 1 else names[-1])

# print(target_modules) # {'project_out', 'v_proj', 'project_in', 'q_proj', 'fc2', 'fc1', 'k_proj', 'out_proj'}
# target_modules
# - q_proj, k_proj, v_proj: Attention의 Q, K, V 벡터 생성.
# - fc1, fc2: Feed-Forward Network로, Transformer 레이어 내 비선형성을 추가.
# - project_in, project_out: Transformer의 입력과 출력을 위한 차원 변환.
# - out_proj: Attention의 출력을 다음 레이어로 전달하기 위해 변환.
# Q와 V가 모델의 표현을 대표하므로, LoRA 목적에 부합함.

if "lm_head" in target_modules:
    target_modules.remove("lm_head")

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=lora_r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    target_modules=["q_proj", "v_proj"],
)

model = get_peft_model(model, peft_config)

# Trainer 설정 및 실행
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    args=SFTConfig(
        output_dir=f"/tmp/{project_name}/rank-{lora_r}", max_seq_length=MAX_LENGTH
    ),
    formatting_func=formatting_prompts_func,
    data_collator=collator,
)
trainer.train()

# 메모리 점유율
print("Max Alloc:", round(torch.cuda.max_memory_allocated(0) / 1024**3, 1), "GB")

wandb.finish()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/677 [00:00<?, ?B/s]

...

Map:   0%|          | 0/20022 [00:00<?, ? examples/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
500,1.908300
1000,1.752400
1500,1.712600
2000,1.684000
2500,1.652300
3000,1.634500
3500,1.608000
4000,1.600200
4500,1.619100
5000,1.594500


Max Alloc: 6.8 GB


train/epoch,▁▁▂▂▃▃▄▄▅▅▆▆▇▇██
train/global_step,▁▁▂▂▃▃▄▄▅▅▆▆▇▇██
train/grad_norm,▁▂▃▂▅▃▄▅▆█▅▄▂▅▇
train/learning_rate,██▇▇▆▅▅▄▄▃▃▃▂▁▁
train/loss,█▅▄▃▃▂▂▂▂▂▁▁▁▁▁
total_flos,2.643172206280704e+16
train/epoch,3
train/global_step,7509
train/grad_norm,1.03595
train/learning_rate,0.0
train/loss,1.565
